In [ ]:
### Universidad de Costa Rica
### Tarea 4 - IE0405: Procesos aleatorios
### Mauricio Brenes Cerdas, B41105

### Inicio del código

### Paquetes requeridos para realizar la tarea
from scipy import stats
from scipy import signal
from scipy import integrate
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt


    
### Se importan los valores 

datos = np.genfromtxt("bits10k.csv",dtype=float, delimiter = ',')
#print(datos)
#print(len(datos))

#### Parte 1 

### Frecuencia de señal portadora en Hertz

f=5000

### Número de bits ingresados

N= 10000

### Periodo de la señal en ms

T=1/f 

### Se define el nímero de puntos de muestreo

P=50

### Se definen los puntos de muestreo por periodo periodo

tp=np.linspace(0,T,P)

### Forma de onda portadora

sin=np.sin(2*np.pi*f*tp)

### Gráfica de la portadora

plt.plot(tp,sin)
plt.xlabel('Tiempo(s)')
plt.title('Onda portadora')
plt.figure(1)
plt.savefig('Onda.png')

### Frecuencia de muestreo

fs= P/T #500 kHz

### Vector temporal para la señal

t = np.linspace(0,N*T,N*P)

### Inicializador

senal= np.zeros(t.shape)

### Se crea la señal modulada BPSK

for i,b in enumerate(datos):
    senal[i*P:(i+1)*P]=np.cos(np.dot(2 * np.pi * fs, T) + np.pi * (b - 1) + np.pi / 4)*sin

### Gráfica de la señal BPSK
plt.figure(2)
plt.plot(senal[0:10*P])
plt.xlabel('Tiempo(s)')
plt.title('Señal modulada BSKP')
plt.savefig('Modulada.png')
plt.show()


####  Parte 2  

### Se define la potencia instantánea

Pinst = senal**2

### Potencia promedio a partir de la potencia instantánea (W)

Ps = integrate.trapz(Pinst, t) / (N * T)
print('La potencia promedio de la señal es:',Ps)

####  Parte 3  

SNR_range = [-2,-1,0,1,2,3]
BER = np.zeros(6)
for i in range(6):
    SNR = SNR_range[i] 

### Potencia del ruido para SNR

Pn = Ps / (10**(SNR / 10))

### Desviación estándar del ruido

sigma = np.sqrt(Pn)

### Se crea la funciónpara añadir ruido a la señal (Pn = sigma^2)

ruido = np.random.normal(0, sigma, senal.shape)

### Señal nueva ya con ruido

Rx = senal + ruido

### Se grafica esta señal con ruido con los primeros bits añadidos

pb = 5
plt.figure()
plt.plot(Rx[0:pb*P])
plt.title('Señal con ruido')
plt.savefig('Ruido.png')
plt.show()


####  PARTE 4 

### Densidad antes de añadir ruido

fw, PSD = signal.welch(senal, fs, nperseg=1024)
plt.figure()
plt.semilogy(fw, PSD)
plt.xlabel('Frecuencia - Hz')
plt.title('Densidad espectral de potencia')
plt.savefig('DenAntes.png')
plt.show()

### Después del canal ruidoso

fw, PSD = signal.welch(Rx, fs, nperseg=1024)
plt.figure()
plt.semilogy(fw, PSD)
plt.xlabel('Frecuencia - Hz')
plt.title('Densidad espectral de potencia')
plt.savefig('DenDespues.png')
plt.show()

###  PARTE 5

### Energía de la señal original

Es = np.sum(sin**2)

### Inicialización del vector de bits recibidos

bitsRx = np.zeros(datos.shape)

### Se decodifica la señal

for k, b in enumerate(datos):
    Ep = np.sum(Rx[k*P:(k+1)*P] * sin)
    if Ep > Es/2:
        bitsRx[k] = 1
    else:
        bitsRx[k] = 0
        
### Se calcula el error

err = np.sum(np.abs(datos - bitsRx))
BER = err/N

print('Hay un total de {} errores en {} bits para una tasa de error de {}.'.format(err, N, BER))


### Parte 6

### Gráfica de SNR vs BER

SNR_range = [-2,-1,0,1,2,3]
BER = [BER,BER,BER,BER,BER,BER]

plt.figure()
plt.plot(BER,SNR_range)
plt.title('BER vs SNR')
plt.savefig('BERvsSNR.png')
plt.show()

